In [1]:
import json
import re

import pandas as pd
import numpy as np

**Reading Data Files**

In [2]:
building_meta_df =  pd.read_csv("buildings.csv")
building_info_df = pd.read_csv("building_info.csv")
building_price_history_df = pd.read_csv("building_pricing_history.csv")
building_unit_df = pd.read_csv("building_unit_data.csv")

**Listing Column Names with Null Values Count**

In [3]:
building_meta_df.isnull().sum()

id                0
name              0
address           0
city              0
state             0
description     634
lat               3
lng               3
created_at        0
zip            3602
type           7284
cooperate         0
summary        7007
company_id     1352
dtype: int64

In [4]:
building_info_df.isna().sum()

id                                  0
company                          1165
website                          5394
phone                             809
email                            7024
pet_policy                       1866
amenities                        1446
neighborhood_name                4827
neighborhood_area                6465
washer_dryer                     5607
no_of_units                      6897
balcony                          5292
parking                          7029
short_term_lease                 7145
tour_sunday                      6995
cooperation_percentage              0
concession_months_additional        0
scraper_complete                 7044
website_reachable                7220
website_needs_ui_navigation      7220
website_data_in_images           7220
data_source                      7044
year_built                       6935
nearest_grocery_store            7097
nearest_convenience_store        7096
nearest_public_transportation    7099
concession_0

In [5]:
building_price_history_df.isna().sum()

id                      0
price              269271
available_date    3573698
created_on              0
dtype: int64

In [6]:
building_unit_df.isna().sum()

id                     0
building_id            0
floor               1762
unit_number        38149
beds                   0
baths                  0
sqft                   0
price                  0
available_date    175790
created_on             0
updated_on        142204
scraped_on             0
scrape_id              0
floorplan          26704
net_price          81787
price_change      419001
dtype: int64

**Listing DataFrame Shapes**

In [7]:
building_meta_df.shape

(7315, 14)

In [8]:
building_info_df.shape

(7220, 49)

In [9]:
building_price_history_df.shape

(3924626, 4)

In [10]:
building_unit_df.shape

(421620, 16)

**Merging building_meta_df, building_info_df and building_unit_df**

In [11]:
merged_df = pd.merge(building_meta_df, building_info_df, left_on='id', right_on='id', how='left')
merged_df.rename(columns={'id': 'building_id'}, inplace=True)
merged_df = pd.merge(merged_df, building_unit_df, left_on='building_id', right_on='building_id', how='inner')

In [12]:
merged_df.shape

(421620, 77)

**Calculating Average Price neighborhood_name**

In [13]:
merged_df['average_price_by_neighborhood'] = merged_df.groupby(['neighborhood_name'])['price'].transform('mean')

**Parse nearest_public_transportation into public_transportation and public_transportation_time**

In [14]:
def parse_public_transportation(row):
    try:
        transportation = json.loads(row['nearest_public_transportation'])[0]
        transportation = re.sub(r'\(.*\)|\d+[\s-]+(?:min).*$', '', transportation)
        return re.sub(r'[\s-]+$', '', transportation)
    except:
        return

def parse_public_transportation_time(row):
    try:
        transportation = json.loads(row['nearest_public_transportation'])[0]
        time = re.findall(r'(\d+)[\s-]+min', transportation)
        return time and int(time[0]) or None
    except:
        return

In [15]:
merged_df['public_transportation'] = merged_df.apply(parse_public_transportation, axis=1)
merged_df['public_transportation_time'] = merged_df.apply(parse_public_transportation_time, axis=1)

**Parse nearest_convenience_store into convenience_store and convenience_store_time**

In [16]:
def parse_convenience_store_time(row):
    times = []
    try:
        for _store in json.loads(row['nearest_convenience_store']):
            time = re.findall(r'(\d+)[\s-]+min', _store)
            
            if not time:
                continue
            
            times.append(int(time[0]))

        return min(times)
    except:
        return
    
def parse_convenience_store(row):
    min_time = parse_convenience_store_time(row)
    try:
        for _store in json.loads(row['nearest_convenience_store']):
            time = re.findall(r'(\d+)[\s-]+min', _store)
            time = time and int(time[0]) or None
            
            if time != min_time:
                continue

            return re.sub(r'[\s-]+$', '', re.sub(r'\(.*\)|\d+[\s-]+(?:min).*$', '', _store))
        return json.loads(row['nearest_convenience_store'])[0]
    except:
        return 

In [17]:
merged_df['convenience_store'] = merged_df.apply(parse_convenience_store, axis=1)
merged_df['convenience_store_time'] = merged_df.apply(parse_convenience_store_time, axis=1)

**Parse nearest_grocery_store into grocery_store and grocery_store_time**

In [18]:
def parse_grocery_store_time(row):
    times = []
    try:
        for _store in json.loads(row['nearest_grocery_store']):
            time = re.findall(r'(\d+)[\s-]+min', _store)
            
            if not time:
                continue
            
            times.append(int(time[0]))

        return min(times)
    except:
        return
    
def parse_grocery_store(row):
    min_time = parse_grocery_store_time(row)
    try:
        for _store in json.loads(row['nearest_grocery_store']):
            time = re.findall(r'(\d+)[\s-]+min', _store)
            time = time and int(time[0]) or None
            
            if time != min_time:
                continue

            return re.sub(r'[\s-]+$', '', re.sub(r'\(.*\)|\d+[\s-]+(?:min).*$', '', _store))
        return json.loads(row['nearest_grocery_store'])[0]
    except:
        return 

In [19]:
merged_df['grocery_store'] = merged_df.apply(parse_grocery_store, axis=1)
merged_df['grocery_store_time'] = merged_df.apply(parse_grocery_store_time, axis=1)

**Map unit_0_balcony, unit_1_balcony, unit_2_balcony, unit_2_balcony to unit_balcony**

In [20]:
def map_balcony(row):
    
    if row['unit_0_balcony'] == 'Yes' and row['beds'] == 0:
        return 1
    
    if row['unit_1_balcony'] == 'Yes' and row['beds'] == 1:
        return 1
    
    if row['unit_2_balcony'] == 'Yes' and row['beds'] == 2:
        return 1
    
    if row['unit_3_balcony'] == 'Yes' and row['beds'] == 3:
        return 1
      
    if row['beds'] > 3:
        
        if 'Yes' in [row['unit_0_balcony'],
                     row['unit_1_balcony'],
                     row['unit_2_balcony'],
                     row['unit_3_balcony']]:
            return 1
        return 0
    
    return 0

In [21]:
merged_df['unit_balcony'] = merged_df.apply(lambda row: map_balcony(row), axis=1)

**Map concession_0_bed, concession_1_bed, concession_2_bed, concession_3_bed to concession_beds**

In [22]:
def map_concession(row):
    
    if row['concession_0_bed'] == 1 and row['beds'] == 0:
        return 1
    
    if row['concession_1_bed'] == 1 and row['beds'] == 1:
        return 1
    
    if row['concession_2_bed'] == 1 and row['beds'] == 2:
        return 1
    
    if row['concession_3_bed'] == 1 and row['beds'] == 3:
        return 1
    
    if row['beds'] > 3:
        if 1 in [row['concession_0_bed'],
                 row['concession_1_bed'],
                 row['concession_2_bed'],
                 row['concession_3_bed']]:
            return 1
        return 0
    
    return 0

In [23]:
merged_df['concession_beds'] = merged_df.apply(lambda row: map_concession(row), axis=1)

**Drop extra columns**

In [24]:
to_drop = ['scraped_on', 
           'scrape_id', 
           'hotspot_description',
           'website_data_in_images',
           'website_needs_ui_navigation', 
           'website_reachable',
           'scraper_complete',
           'data_source',
           'leasing_type',
           'architect',
           'aptamigo',
           'hotspot_features',
           'hotspot_neighbourhood',
           'hotspot',
           'balcony',
           'unit_0_balcony',
           'unit_convert_balcony',
           'unit_1_balcony',
           'unit_2_balcony',
           'unit_3_balcony',
           'concession_title',
           'concession_0_bed',
           'concession_1_bed',
           'concession_2_bed',
           'concession_3_bed',
           'concession_source',
           'nearest_public_transportation',
           'nearest_convenience_store',
           'nearest_grocery_store',
           'unit_number',
           'type',
           'email',
           'updated_on',
           'created_on',
           'created_at',
           'summary',
           'description',
           'address',
           'name',
           'website',
           'phone',
           'email',
           'zip',
           'amenities',
           'no_of_units',
          ]

In [25]:
merged_df.drop(columns=to_drop, inplace=True)
merged_df.shape

(421620, 42)

**Handling Missing Value**

In [27]:
print(merged_df.washer_dryer.unique())
merged_df['washer_dryer'].fillna('No', inplace=True)

['No' 'Yes' nan 'Select Units']


In [28]:
print(merged_df.parking.unique())
merged_df['parking'].fillna('No', inplace=True)

[nan 'Yes' 'No' 'Garage Parking' 'Attached Parking' 'Offsite Parking'
 'Select Units' 'Assigned Parking']


In [29]:
print(merged_df.pool.unique())
merged_df['pool'].fillna('No', inplace=True)

['Yes' nan 'No']


In [30]:
print(merged_df.tour_sunday.unique())
merged_df['tour_sunday'].fillna('No', inplace=True)

[False nan True]


In [35]:
print(merged_df.short_term_lease.unique())
merged_df['short_term_lease'].fillna('No', inplace=True)

[nan 'No' 'Sometimes' 'Yes' 'During Peak Season']


In [36]:
merged_df.isna().sum()

building_id                           0
city                                  0
state                                 0
lat                                  11
lng                                  11
cooperate                             0
company_id                        10183
company                            9684
pet_policy                        16904
neighborhood_name                253295
neighborhood_area                356398
washer_dryer                          0
parking                               0
short_term_lease                      0
tour_sunday                           0
cooperation_percentage             8908
concession_months_additional       8908
year_built                       372942
concession_additional              8908
concession_update                  8908
flooring                         395517
age                              373192
pool                                  0
id                                    0
floor                              1762
